In [1]:
# IMPORT PACKAGES
import pandas as pd
import sqlalchemy as sa
import pymysql
import os
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cryptography
from sklearn.linear_model import LinearRegression

In [2]:
# CONNECT TO LOCAL FANTASY SQL DATABASE
# DEFINE THE DATABASE CREDENTIALS
user = 'root'
password = 'password123'
host = 'localhost'
port = 3306
database = 'fantasydb'

# PYTHON FUNCTION TO CONNECT TO THE MYSQL DATABASE AND
# RETURN THE SQLACHEMY ENGINE OBJECT
def get_connection():
	return create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port, database
		)
	)

# CONNECT TO SQL DATABASE
try:
    # GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
    engine = get_connection()
    print(
        f"Connection to the {host} for user {user} created successfully.")
except Exception as ex:
    print("Connection could not be made due to the following error: \n", ex)

Connection to the localhost for user root created successfully.


In [3]:
# GET DATAFRAME OF ALL PLAYERS OF *POSITION* IN *YEAR*
def retrieve_players(pos, year):
    with engine.begin() as conn:
        df = pd.read_sql_query(sa.text(
            f'''
            SELECT 
                *
            FROM 
                {pos}_stats_{year}
            '''), conn)
        conn.close()
    return df

# GET THE TOP *RANK* *POSITION* PLAYERS OF *YEAR* AND THEIR *POSITION* STATS FROM PRIOR *YEAR*
def top_x_players_prior_stats(pos, year, rank):
    valid_year = ['2019', '2020', '2021', '2022']
    valid_pos = ['qb', 'wr', 'wr', 'te']
    
    if year not in valid_year:
        return print('Not a valid year: 2019-2022')
    if pos not in valid_pos:
        return print('Not a valid position')
    
    prior_year= str(int(year) -1)
    age_change = 2023 - int(prior_year)
    
    with engine.begin() as conn:
        df = pd.read_sql_query(sa.text(
            f'''
            SELECT
                {pos}_stats_{year}.NAME as NAME1,
                {pos}_stats_{year}.RANK as RANK1,
                ppr_adp_{prior_year}.AGE,
                {pos}_stats_{prior_year}.*
            FROM 
                {pos}_stats_{prior_year}
            INNER JOIN 
                {pos}_stats_{year} 
            ON 
                {pos}_stats_{year}.NAME = {pos}_stats_{prior_year}.NAME
            INNER JOIN
                ppr_adp_{prior_year}
            ON
                {pos}_stats_{prior_year}.NAME = ppr_adp_{prior_year}.NAME
                
            WHERE
                ({pos}_stats_{year}.FPTS / {pos}_stats_{year}.G) >= 16
            ORDER BY
                {pos}_stats_{year}.RANK ASC            
            '''), conn)
        conn.close()
        
        df = df.drop(columns=['index', 'RANK', 'NAME'])
        df["AGE"] = df["AGE"].subtract(age_change)
        df.columns = ['PRIOR_' + str(col) for col in df.columns]
        df = df.rename(columns={'PRIOR_NAME1':'NAME', 'PRIOR_RANK1':'PRIOR_RANK', })
    return df

In [4]:
df1 = top_x_players_prior_stats('wr', '2019', '12')
df2 = top_x_players_prior_stats('wr', '2020', '12')
df3 = top_x_players_prior_stats('wr', '2021', '12')
df4 = top_x_players_prior_stats('wr', '2022', '12')
legendary_wrs = pd.concat([df1, df2, df3, df4], ignore_index=True)

ppr_per_game = round(legendary_wrs['PRIOR_FPTS']/legendary_wrs['PRIOR_G'], 1)
rec_yds_per_game = round(legendary_wrs['PRIOR_REC_YDS']/legendary_wrs['PRIOR_G'], 1)
tgt_per_game = round(legendary_wrs['PRIOR_TGT']/legendary_wrs['PRIOR_G'], 1)
rz_tgt_per_game = round(legendary_wrs['PRIOR_RZ_TGT']/legendary_wrs['PRIOR_G'], 1)


legendary_wrs['PRIOR_REC_YDS_PER_G'] = rec_yds_per_game
legendary_wrs['PRIOR_PPR_POINTS_PER_G'] = ppr_per_game
legendary_wrs['TGT_PER_G'] = tgt_per_game
legendary_wrs['RZ_TGT_PER_G'] = rz_tgt_per_game


legendary_wrs = legendary_wrs[['NAME', 'PRIOR_RANK', 'PRIOR_AGE',
                       'PRIOR_REC_YDS_PER_G', 'PRIOR_PPR_POINTS_PER_G', 'TGT_PER_G', 'RZ_TGT_PER_G',
                       'PRIOR_TGT_SHARE', 'PRIOR_TGT','PRIOR_RZ_TGT']]

desc_wr = legendary_wrs.describe()
legendary_wrs

,NAME,PRIOR_RANK,PRIOR_AGE,PRIOR_REC_YDS_PER_G,PRIOR_PPR_POINTS_PER_G,TGT_PER_G,RZ_TGT_PER_G,PRIOR_TGT_SHARE,PRIOR_TGT,PRIOR_RZ_TGT
0,Michael Thomas,1.0,25.0,87.8,19.7,9.2,1.3,0.288,147.0,21.0
1,Chris Godwin,2.0,22.0,52.6,11.6,5.9,0.6,0.153,95.0,10.0
2,Cooper Kupp,4.0,25.0,70.8,16.9,6.9,0.6,0.101,55.0,5.0
3,Julio Jones,5.0,29.0,104.8,20.4,10.6,0.8,0.281,170.0,13.0
4,Mike Evans,8.0,24.0,95.2,17.8,8.6,0.4,0.223,138.0,6.0
5,DeAndre Hopkins,9.0,26.0,98.2,20.8,10.2,0.6,0.330,163.0,10.0
6,Keenan Allen,11.0,26.0,74.8,16.3,8.5,0.6,0.269,136.0,9.0
7,Julian Edelman,13.0,32.0,70.8,17.3,9.0,0.8,0.194,108.0,9.0
8,Davante Adams,29.0,25.0,92.4,22.0,11.3,1.0,0.275,169.0,15.0
9,Antonio Brown,146.0,30.0,86.5,21.6,11.2,0.8,0.249,168.0,12.0


## Legendary Wide Recievers (2019 - 2022)
In the span of 4 years, there have been 10 instances of wide receivers avergaing 20+ PPR PPG

In [5]:
desc_wr

,PRIOR_RANK,PRIOR_AGE,PRIOR_REC_YDS_PER_G,PRIOR_PPR_POINTS_PER_G,TGT_PER_G,RZ_TGT_PER_G,PRIOR_TGT_SHARE,PRIOR_TGT,PRIOR_RZ_TGT
count,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000
mean,14.068182,25.590909,76.125000,16.950000,8.534091,0.970455,0.222659,126.022727,14.500000
std,23.583003,2.661685,15.719062,3.482414,1.791445,0.453708,0.062622,37.181821,7.693247
min,1.000000,21.000000,41.800000,11.400000,4.800000,0.300000,0.079000,44.000000,2.000000
25%,4.000000,24.000000,66.000000,14.675000,6.975000,0.600000,0.187750,98.750000,9.000000
50%,7.500000,26.000000,71.900000,16.550000,8.800000,0.850000,0.235500,131.500000,13.000000
75%,12.250000,27.000000,87.575000,18.575000,10.050000,1.200000,0.272750,157.000000,18.250000
max,146.000000,32.000000,114.500000,25.900000,11.300000,2.200000,0.330000,191.000000,37.000000


In [6]:
# GET DATAFRAME OF RUNNING BACKS FROM *YEAR* WHO HAVE LEGENDARY POTENTIAL
# i.e. STATS BETTER THAN LEGENDARY RUNNING BACK AVERAGE STATS MINUS ONE STANDARD DEVIATION
def retrieve_breakout_wrs(year):
    with engine.begin() as conn:
        df = pd.read_sql_query(sa.text(
            f'''
            SELECT
                wr_stats_{year}.*,
                ppr_adp_{str(int(year)+1)}.ADP,
                ppr_adp_{str(int(year)+1)}.AGE
            FROM 
                wr_stats_{year}
            INNER JOIN 
                ppr_adp_{str(int(year)+1)} 
            ON 
                wr_stats_{year}.NAME = ppr_adp_{str(int(year)+1)}.NAME 
            WHERE
                wr_stats_{year}.G > 8   
            AND
                ppr_adp_{str(int(year)+1)}.AGE <= 27
            AND 
                (wr_stats_{year}.REC_YDS / wr_stats_{year}.G)  > {desc_wr['PRIOR_REC_YDS_PER_G'][1]-desc_wr['PRIOR_REC_YDS_PER_G'][2]}
            AND 
                (wr_stats_{year}.TGT / wr_stats_{year}.G)  > {desc_wr['TGT_PER_G'][1]-desc_wr['TGT_PER_G'][2]}
            
            ORDER BY
                ppr_adp_{str(int(year)+1)}.ADP
            ASC
            '''), conn)
        conn.close()
        
        df = df[['NAME', 'RANK', 'ADP']]

    return df

# GET DATAFRAME COMPARING THE OVERALL RANK OF RUNNING BACKS FROM *retrieve_breakout_wrs()* 
# FROM *YEAR* AND THEIR OVERALL RANK THE FOLLOWING YEAR
def compare_breakout_wr(year):
    df1 = retrieve_breakout_wrs(year)
    df2 = retrieve_players('wr',str(int(year)+1))
    df3 = df1.merge(df2, on='NAME')
    df3 = df3[['NAME', 'RANK_x', 'ADP', 'RANK_y']].rename(columns={
        'RANK_x': year + '_RANK', 'ADP': year + '_ADP', 'RANK_y': str(int(year)+1)+'_RANK'
    })
    df3['CHANGE'] = df3[ year + '_RANK'] - df3[str(int(year)+1)+'_RANK']
    df3['IMPROVE'] = df3['CHANGE'] >= 0
    df3['TOP_10'] = df3[str(int(year)+1)+'_RANK'] <= 10
    df3 = df3[['NAME', year+'_ADP', year+'_RANK', str(int(year)+1)+'_RANK', 'CHANGE', 'IMPROVE', 'TOP_10']]
    return df3

In [7]:
# COMPARE wrS WITH LEGENDARY POTENTIAL FROM 2018 TO 2021
br_wr_2018 = compare_breakout_wr('2018')
br_wr_2019 = compare_breakout_wr('2019')
br_wr_2020 = compare_breakout_wr('2020')
br_wr_2021 = compare_breakout_wr('2021')

In [8]:
br_wr_2018

,NAME,2018_ADP,2018_RANK,2019_RANK,CHANGE,IMPROVE,TOP_10
0,JuJu Smith-Schuster,12.9,9.0,66.0,-57.0,False,False


In [9]:
br_wr_2019

,NAME,2019_ADP,2019_RANK,2020_RANK,CHANGE,IMPROVE,TOP_10
0,Chris Godwin,21.5,2.0,30.0,-28.0,False,False
1,DJ Moore,32.6,21.0,17.0,4.0,True,False
2,DJ Chark Jr.,47.5,16.0,48.0,-32.0,False,False
3,Courtland Sutton,58.1,17.0,172.0,-155.0,False,False
4,Michael Gallup,72.2,20.0,36.0,-16.0,False,False


In [10]:
br_wr_2020

,NAME,2020_ADP,2020_RANK,2021_RANK,CHANGE,IMPROVE,TOP_10
0,DK Metcalf,18.5,5.0,10.0,-5.0,False,True
1,Justin Jefferson,22.6,6.0,4.0,2.0,True,True
2,A.J. Brown,24.1,9.0,32.0,-23.0,False,False
3,Terry McLaurin,28.6,23.0,24.0,-1.0,False,False
4,Diontae Johnson,44.8,26.0,12.0,14.0,True,False
5,Chris Godwin,46.8,30.0,19.0,11.0,True,False
6,Brandon Aiyuk,54.5,33.0,34.0,-1.0,False,False
7,DJ Moore,59.1,17.0,18.0,-1.0,False,False


In [11]:
br_wr_2021

,NAME,2021_ADP,2021_RANK,2022_RANK,CHANGE,IMPROVE,TOP_10
0,Justin Jefferson,4.9,4.0,1.0,3.0,True,True
1,Ja'Marr Chase,8.8,3.0,12.0,-9.0,False,False
2,CeeDee Lamb,17.9,14.0,6.0,8.0,True,True
3,Deebo Samuel,19.6,2.0,34.0,-32.0,False,False
4,Michael Pittman Jr.,24.7,16.0,27.0,-11.0,False,False
5,A.J. Brown,30.5,32.0,4.0,28.0,True,True
6,Tee Higgins,32.4,17.0,17.0,0.0,True,False
7,DJ Moore,36.1,18.0,20.0,-2.0,False,False
8,Terry McLaurin,38.0,24.0,14.0,10.0,True,False
9,Diontae Johnson,43.5,12.0,46.0,-34.0,False,False


## Observations


In [12]:
df = retrieve_breakout_wrs('2022')
df.loc[-1] = ['Jahan Dotson', 51, 73]  # adding a row
df.index = df.index + 1  # shifting index

In [13]:
potential_wr_2023 = df
potential_wr_2023

,NAME,RANK,ADP
1,Justin Jefferson,1.0,1
2,Ja'Marr Chase,12.0,3
3,CeeDee Lamb,6.0,11
4,A.J. Brown,4.0,12
5,Amon-Ra St. Brown,9.0,16
6,Garrett Wilson,21.0,18
7,Jaylen Waddle,7.0,23
8,DeVonta Smith,10.0,25
9,Chris Olave,23.0,27
10,Tee Higgins,17.0,30
